In [1]:
!nvidia-smi

Sat May 22 10:25:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.27       Driver Version: 466.27       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:08:00.0  On |                  N/A |
| 33%   29C    P8    13W / 320W |   1361MiB / 10240MiB |     28%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from bertopic import BERTopic
import pandas as pd
import gensim
import nltk
import re
from sentence_transformers import SentenceTransformer
import string
from gensim.utils import simple_preprocess
from nltk.stem.snowball import SnowballStemmer
from pprint import pprint
import gensim.corpora as corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pEhne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop_words = stopwords.words('german')

In [5]:
tweets = pd.read_csv("D:\Tweets_bis_Apr4.csv")

In [6]:
tweets['hashtag'] = tweets['text'].apply(lambda x: re.findall(r'#(\w+)', x))

In [7]:
# def remove_stopwords(texts):
#     return [word for word in texts if word not in stop_words and word not in string.punctuation]

In [7]:
tweets['text_processed'] = tweets['text'].map(lambda x: re.sub(r'^RT[\s]+', '', x)) 
tweets['text_processed'] = tweets['text_processed'].map(lambda x: re.sub(r'https?:\/\/.*[\r\n]*', '', x))
tweets['text_processed'] = tweets['text_processed'].map(lambda x: re.sub(r'[^A-Za-zÀ-ž ]', '', x))
##tweets['text_processed'] = tweets['text_processed'].map(lambda x: word_tokenize(x)) 

#tweets['text_processed'] = tweets['text_processed'].map(lambda x: x.encode('ascii', 'ignore').decode('utf-8', 'ignore')) #unicodedata.normalize('NFKD', x)

In [9]:
#tweets['text_processed'] = tweets['text_processed'].map(lambda x: [word for word in x if  word not in stop_words and word not in string.punctuation])

In [8]:
#sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cuda")
topic_model = BERTopic(language='german') #embedding_model=sentence_model,
topics, _ = topic_model.fit_transform(tweets['text_processed'])

In [13]:
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [14]:
topic_model = BERTopic(language='multilingual')
topics, _ = topic_model.fit_transform(docs)


100%|██████████| 1.01G/1.01G [01:17<00:00, 13.0MB/s]


KeyboardInterrupt: 

In [ ]:
topic_model.visualize_topics()

In [8]:
# def deEmojify(text):
#     regrex_pattern = re.compile(pattern = "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags = re.UNICODE)
#     return regrex_pattern.sub(r'',text)
# def sent_to_words(sentences):
#     for sentence in sentences:
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


In [9]:
#p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.RESERVED, p.OPT.NUMBER, p.OPT.EMOJI, p.OPT.SMILEY)
#tweets['text_processed'] = tweets['text'].map(lambda x: p.clean(x))

In [10]:
tweet_data = tweets.text_processed.values.tolist()
#tweet_words = list(sent_to_words(tweet_data))# remove stop words
#tweet_words = remove_stopwords(tweet_words)

In [11]:
id2word = corpora.Dictionary(tweet_words)# Create Corpus
texts = tweet_words
corpus = [id2word.doc2bow(text) for text in texts]

In [12]:
# number of topics
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.011*"amp" + 0.008*"fur" + 0.008*"heute" + 0.005*"uber" + 0.005*"bundestag" + 0.004*"the" + 0.003*"deutschland" + 0.003*"spd" + 0.003*"tb" + 0.003*"berlin"'), (1, '0.012*"fur" + 0.010*"heute" + 0.008*"amp" + 0.004*"geht" + 0.003*"corona" + 0.003*"mal" + 0.003*"uber" + 0.003*"gerade" + 0.003*"wurde" + 0.003*"ja"'), (2, '0.020*"fur" + 0.006*"amp" + 0.004*"bundestag" + 0.004*"berlin" + 0.004*"uber" + 0.004*"tb" + 0.003*"mehr" + 0.003*"gut" + 0.003*"beim" + 0.003*"afd"'), (3, '0.009*"fur" + 0.005*"mussen" + 0.005*"the" + 0.004*"uber" + 0.004*"heute" + 0.004*"mehr" + 0.003*"geht" + 0.003*"menschen" + 0.003*"amp" + 0.003*"immer"'), (4, '0.010*"uber" + 0.009*"fur" + 0.007*"heute" + 0.005*"cdu" + 0.004*"geht" + 0.004*"gibt" + 0.003*"mehr" + 0.003*"konnen" + 0.003*"bundestag" + 0.003*"bundesregierung"'), (5, '0.032*"fur" + 0.006*"mehr" + 0.005*"bundestag" + 0.005*"uber" + 0.005*"deutschland" + 0.005*"menschen" + 0.004*"fdp" + 0.004*"heute" + 0.004*"immer" + 0.004*"amp"'), (6, '0.017*"fur

In [4]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))


def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pEhne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pEhne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
